# Capitulo 6: Precognicion (Pensar Paso a Paso)

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu API key y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

# Corregir la ruta de importacion para las pistas
import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Leccion

Si alguien te despertara y de inmediato comenzara a hacerte varias preguntas complicadas que tuvieras que responder de inmediato, como te iria? Probablemente no tan bien como si te dieran tiempo para **pensar tu respuesta primero**.

Adivina que? Claude es igual.

**Darle tiempo a Claude para pensar paso a paso a veces lo hace mas preciso**, particularmente para tareas complejas. Sin embargo, **el pensamiento solo cuenta cuando es en voz alta**. No puedes pedirle a Claude que piense pero que solo muestre la respuesta - en ese caso, realmente no ha ocurrido ningun pensamiento.

### Ejemplos

En el prompt a continuacion, es claro para un lector humano que la segunda oracion contradice la primera. Pero **Claude toma la palabra "unrelated" demasiado literalmente**.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

Para mejorar la respuesta de Claude, vamos a **permitir que Claude piense las cosas primero antes de responder**. Hacemos eso literalmente detallando los pasos que Claude debe seguir para procesar y pensar sobre su tarea. Junto con un toque de prompt de rol, esto le permite a Claude comprender la resena mas profundamente.

In [ ]:
# Prompt de sistema
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude a veces es sensible al orden**. Este ejemplo esta en la frontera de la capacidad de Claude para entender texto con matices, y cuando intercambiamos el orden de los argumentos del ejemplo anterior para que negativo sea primero y positivo sea segundo, esto cambia la evaluacion general de Claude a positiva.

En la mayoria de las situaciones (pero no todas, confusamente), **Claude es mas propenso a elegir la segunda de dos opciones**, posiblemente porque en sus datos de entrenamiento de la web, las segundas opciones tenian mas probabilidad de ser correctas.

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

**Dejar que Claude piense puede cambiar la respuesta de Claude de incorrecta a correcta**. Es asi de simple en muchos casos donde Claude comete errores!

Veamos un ejemplo donde la respuesta de Claude es incorrecta para ver como pedirle a Claude que piense puede corregir eso.

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

Vamos a corregir esto pidiendole a Claude que piense paso a paso, esta vez en etiquetas `<brainstorm>`.

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 6.1 - Clasificacion de Correos](#ejercicio-61---clasificacion-de-correos)
- [Ejercicio 6.2 - Formato de Clasificacion de Correos](#ejercicio-62---formato-de-clasificacion-de-correos)

### Ejercicio 6.1 - Clasificacion de Correos
En este ejercicio, le indicaremos a Claude que clasifique correos electronicos en las siguientes categorias:
- (A) Pregunta de pre-venta
- (B) Articulo roto o defectuoso
- (C) Pregunta de facturacion
- (D) Otro (por favor explique)

Para la primera parte del ejercicio, cambia el `PROMPT` para **hacer que Claude genere la clasificacion correcta y SOLO la clasificacion**. Tu respuesta necesita **incluir la letra (A - D) de la opcion correcta, con los parentesis, asi como el nombre de la categoria**.

Consulta los comentarios junto a cada correo en la lista `EMAILS` para saber a que categoria debe clasificarse cada correo.

In [ ]:
# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = """Please classify this email as either green or blue: {email}"""

# Pre-llenado para la respuesta de Claude, si hay
PREFILL = ""

# Contenido variable almacenado como una lista
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Articulo roto o defectuoso
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pregunta de pre-venta O (D) Otro (por favor explique)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Pregunta de facturacion
    "How did I get here I am not good with computer.  Halp." # (D) Otro (por favor explique)
]

# Categorizaciones correctas almacenadas como una lista de listas para acomodar la posibilidad de multiples categorizaciones correctas por correo
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Diccionario de valores de cadena para cada categoria para usar en la calificacion con regex
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterar a traves de la lista de correos
for i,email in enumerate(EMAILS):
    
    # Sustituir el texto del correo en la variable de marcador de posicion del correo
    formatted_prompt = PROMPT.format(email=email)
   
    # Obtener la respuesta de Claude
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Calificar la respuesta de Claude
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Imprimir la respuesta de Claude
    print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
    print("TURNO DEL USUARIO")
    print(formatted_prompt)
    print("\nTURNO DEL ASISTENTE")
    print(PREFILL)
    print("\n------------------------------------- Respuesta de Claude -------------------------------------")
    print(response)
    print("\n------------------------------------------ CALIFICACION ------------------------------------------")
    print("Este ejercicio se ha resuelto correctamente:", grade, "\n\n\n\n\n\n")

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

Aun atascado? Ejecuta la celda de abajo para ver una solucion de ejemplo.

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### Ejercicio 6.2 - Formato de Clasificacion de Correos
En este ejercicio, vamos a refinar el output del prompt anterior para obtener una respuesta formateada exactamente como la queremos.

Usa tu tecnica favorita de formato de output para hacer que Claude envuelva SOLO la letra de la clasificacion correcta en etiquetas `<answer></answer>`. Por ejemplo, la respuesta al primer correo debe contener la cadena exacta `<answer>B</answer>`.

Consulta los comentarios junto a cada correo en la lista `EMAILS` si olvidas que letra de categoria es correcta para cada correo.

In [ ]:
# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = """Please classify this email as either green or blue: {email}"""

# Pre-llenado para la respuesta de Claude, si hay
PREFILL = ""

# Contenido variable almacenado como una lista
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Articulo roto o defectuoso
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pregunta de pre-venta O (D) Otro (por favor explique)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Pregunta de facturacion
    "How did I get here I am not good with computer.  Halp." # (D) Otro (por favor explique)
]

# Categorizaciones correctas almacenadas como una lista de listas para acomodar la posibilidad de multiples categorizaciones correctas por correo
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Diccionario de valores de cadena para cada categoria para usar en la calificacion con regex
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterar a traves de la lista de correos
for i,email in enumerate(EMAILS):
    
    # Sustituir el texto del correo en la variable de marcador de posicion del correo
    formatted_prompt = PROMPT.format(email=email)
   
    # Obtener la respuesta de Claude
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Calificar la respuesta de Claude
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Imprimir la respuesta de Claude
    print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
    print("TURNO DEL USUARIO")
    print(formatted_prompt)
    print("\nTURNO DEL ASISTENTE")
    print(PREFILL)
    print("\n------------------------------------- Respuesta de Claude -------------------------------------")
    print(response)
    print("\n------------------------------------------ CALIFICACION ------------------------------------------")
    print("Este ejercicio se ha resuelto correctamente:", grade, "\n\n\n\n\n\n")

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### Felicidades!

Si has resuelto todos los ejercicios hasta este punto, estas listo para pasar al siguiente capitulo. Feliz prompting!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompt mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt de sistema
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))